In [75]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os

acropolis_path = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Database/Old"

In [76]:
df = pl.scan_parquet(os.path.join(acropolis_path, "old_db_measurements.parquet"))

df = df.rename({"date": "creation_timestamp",
                "system name": "system_name",
                "CO2_raw": "gmp343_raw",
                "CO2_compensated": "gmp343_compensated",
                "CO2_filtered": "gmp343_filtered",
                "chamber_temperature": "gmp343_temperature",
                "speed_avg": "wxt532_speed_avg",
                "speed_min": "wxt532_speed_min",
                "speed_max": "wxt532_speed_max",
                "direction_avg": "wxt532_direction_avg",
                "direction_min": "wxt532_direction_min",
                "direction_max": "wxt532_direction_max",
                "last_update_time": "wxt532_last_update_time",
                "cpu_usage": "raspi_cpu_usage",
                "cpu_temperature": "raspi_cpu_temperature",
                "disk_usage": "raspi_disk_usage",
                "enclosure_humidity": "enclosure_bme280_humidity",
                "enclosure_pressure": "enclosure_bme280_pressure",
                "mainboard_temperature": "enclosure_bme280_temperature",})

df.tail().collect()

integer,creation_timestamp,system_name,variant,gmp343_raw,gmp343_compensated,gmp343_filtered,sht45_humidity,bme280_humidity,bme280_pressure,sht45_temperature,bme280_temperature,gmp343_temperature,gas_bottle_id,wxt532_speed_avg,wxt532_speed_max,wxt532_speed_min,wxt532_direction_avg,wxt532_direction_max,wxt532_direction_min,wxt532_last_update_time,raspi_cpu_usage,raspi_disk_usage,raspi_cpu_temperature,enclosure_bme280_humidity,enclosure_bme280_pressure,enclosure_bme280_temperature
i64,"datetime[μs, UTC]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
11166354,2023-09-21 11:59:55.650 UTC,"""tum-esm-midcos…","""measurement""",342.4,405.8,405.8,31.71,29.84,904.03,32.34,32.14,33.7,null,null,null,null,null,null,null,null,null,null,null,null,null,null
11166355,2023-09-21 11:59:56.220 UTC,"""tum-esm-midcos…","""measurement""",347.2,402.5,402.5,26.43,23.82,927.65,33.37,33.17,34.8,null,null,null,null,null,null,null,null,null,null,null,null,null,null
11166356,2023-09-21 11:59:56.750 UTC,"""tum-esm-midcos…","""measurement""",370.0,426.6,426.6,29.62,28.44,923.37,30.79,30.65,32.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
11166357,2023-09-21 11:59:57.310 UTC,"""tum-esm-midcos…","""measurement""",364.0,422.5,422.5,29.22,25.44,922.38,30.73,30.3,31.3,null,null,null,null,null,null,null,null,null,null,null,null,null,null
11166358,2023-09-21 11:59:57.400 UTC,"""tum-esm-midcos…","""measurement""",373.7,433.9,433.9,27.63,24.4,923.61,32.11,31.81,33.3,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [77]:
# select calibration data and create new colomn with cal_*

df = df.with_columns(pl.when(pl.col("variant")== "calibration").then(pl.col("gmp343_raw")).otherwise(None).alias("cal_gmp343_raw"))

df = df.with_columns(pl.when(pl.col("variant")== "calibration").then(pl.col("gmp343_compensated")).otherwise(None).alias("cal_gmp343_compensated"))

df = df.with_columns(pl.when(pl.col("variant")== "calibration").then(pl.col("gmp343_filtered")).otherwise(None).alias("cal_gmp343_filtered"))

#delete calibration readings from columns gmp343_*

df = df.with_columns(pl.when(pl.col("variant")== "measurement").then(pl.col("gmp343_raw")).otherwise(None))

df = df.with_columns(pl.when(pl.col("variant")== "measurement").then(pl.col("gmp343_compensated")).otherwise(None))

df = df.with_columns(pl.when(pl.col("variant")== "measurement").then(pl.col("gmp343_filtered")).otherwise(None))

df = df.select(pl.exclude("integer")) 

df.filter(pl.col("variant") == "calibration").head().collect()

creation_timestamp,system_name,variant,gmp343_raw,gmp343_compensated,gmp343_filtered,sht45_humidity,bme280_humidity,bme280_pressure,sht45_temperature,bme280_temperature,gmp343_temperature,gas_bottle_id,wxt532_speed_avg,wxt532_speed_max,wxt532_speed_min,wxt532_direction_avg,wxt532_direction_max,wxt532_direction_min,wxt532_last_update_time,raspi_cpu_usage,raspi_disk_usage,raspi_cpu_temperature,enclosure_bme280_humidity,enclosure_bme280_pressure,enclosure_bme280_temperature,cal_gmp343_raw,cal_gmp343_compensated,cal_gmp343_filtered
"datetime[μs, UTC]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2023-06-26 14:01:38.950 UTC,"""tum-esm-midcos…","""calibration""",null,null,null,34.29,34.66,956.66,27.9,27.74,28.8,"""991""",null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0
2023-06-26 14:01:48.960 UTC,"""tum-esm-midcos…","""calibration""",null,null,null,34.41,34.83,956.4,27.88,27.75,28.8,"""991""",null,null,null,null,null,null,null,null,null,null,null,null,null,458.8,501.1,501.1
2023-06-26 14:01:58.980 UTC,"""tum-esm-midcos…","""calibration""",null,null,null,34.47,34.9,957.19,27.9,27.76,28.8,"""991""",null,null,null,null,null,null,null,null,null,null,null,null,null,459.6,502.3,502.3
2023-06-26 14:02:08.980 UTC,"""tum-esm-midcos…","""calibration""",null,null,null,34.43,34.84,956.62,27.9,27.75,28.8,"""991""",null,null,null,null,null,null,null,null,null,null,null,null,null,455.3,496.9,496.9
2023-06-26 14:02:18.990 UTC,"""tum-esm-midcos…","""calibration""",null,null,null,34.42,34.84,956.11,27.9,27.75,28.9,"""991""",null,null,null,null,null,null,null,null,null,null,null,null,null,455.4,497.6,497.6


In [79]:
df.sink_parquet(os.path.join(acropolis_path, "old_db_renamed_measurements.parquet"))